In [17]:
import praw, json, os
from datetime import datetime
import pandas as pd

In [2]:
user_agent = 'python:sentiment_analysis:v0.0.1 (by u/Doomsday39)'
reddit = praw.Reddit("bot", user_agent=user_agent)
print(reddit.read_only)

True


In [24]:
smoking_words = ['smoke', 'smoker', 'smoking', 'cigar', 'cigarette', 'weed', 'marijuana', 'drug', 'juul', 'cannabis', 'tobacco', 'opiate', 'vape', 'vaping', 'addiction']
subreddits = [reddit.subreddit('stopsmoking'), reddit.subreddit('quittingsmoking'), reddit.subreddit('QuitVaping'), reddit.subreddit('QuittingJUUL')]
times = ['all', 'hour', 'day', 'week', 'month', 'year']

In [5]:
subs = []
for word in smoking_words:
    for sub in reddit.subreddits.search(word):
        if sub not in subs:
            subs.append(sub)
            print(sub.display_name)

Cigarettes
smoking
trees
BBQ
PopSmoke
hiphopheads
DMT
cigars
stopsmoking
GlobalOffensive
weed
PipeTobacco
Rainbow6
hookah
pics
electronic_cigarette
leaves
saplings
uktrees
AskReddit
Showerthoughts
Drugs
entwives
teenagers
Marijuana
todayilearned
mildlyinteresting
memes
StonerEngineering
see
funny
unpopularopinion
Vaping
MortalKombat
vaporents
dankmemes
circlejerk
worldnews
WorldOfWarships
csgo
Warthunder
woahdude
interestingasfuck
OldSchoolCool
cubancigars
oddlysatisfying
australia
NoStupidQuestions
videos
apexlegends
Knife_Swap
WTF
Petioles
pelletgrills
AdviceAnimals
microgrowery
politics
Jokes
drugscirclejerk
Portland
highdeas
FLMedicalTrees
Traeger
ShittyLifeProTips
BattlefieldV
ArtOfRolling
hempflowers
gifs
IndianEnts
news
eldertrees
Simulated
imaginedragons
blender
VALORANT
AmItheAsshole
quittingsmoking
trashy
reddit.com
Stims
WritingPrompts
explainlikeimfive
FortNiteBR
opiates
SiegeAcademy
bayarea
ausents
oilpen
knifeclub
2007scape
conspiracy
food
DotA2
Crainn
LSD
aww
shittyasksc

In [25]:
all_data = []
submissions = {}
num_of_words = 0
num_of_posts_title_body = 0
num_of_posts_title_url = 0
num_of_posts_per_subreddit = {}
for subreddit in subreddits:
    subreddit_name = subreddit.display_name
    print(subreddit_name + ' processing...')
    num_of_posts_per_subreddit[subreddit_name] = 0
    submissions[subreddit_name] = []
    i = 0
    while num_of_posts_per_subreddit[subreddit_name] < 1000 and i < len(times):
        for submission in subreddit.top(times[i], limit=1000):
            if submission.id not in submissions[subreddit.display_name]:
                if submission.is_self:
                    submissions[subreddit_name].append(submission.id)
                    post = {
                        'subreddit': subreddit_name,
                        'id': submission.id,
                        'title': submission.title,
                        'url': submission.url,
                        'date_created': datetime.fromtimestamp(submission.created_utc).strftime("%m/%d/%Y, %H:%M:%S"),
                        'body': submission.selftext
                    }
                    num_of_words += len(post['body'].split())
                    if not post['body'] and post['url'] and post['title']:
                        num_of_posts_title_url += 1
                    elif post['body'] and post['title']:
                        num_of_posts_title_body += 1
                    num_of_posts_per_subreddit[subreddit_name] += 1
                    all_data.append(post)
                    if num_of_posts_per_subreddit[subreddit_name] == 1000:
                        break
        i += 1

stopsmoking processing...
quittingsmoking processing...
QuitVaping processing...
QuittingJUUL processing...


In [26]:
print(len(all_data))
print(num_of_words)
print(num_of_posts_title_body)
print(num_of_posts_title_url)
for subreddit in num_of_posts_per_subreddit:
    print('{} has {} posts'.format(subreddit, num_of_posts_per_subreddit[subreddit]))

4000
549535
3813
187
stopsmoking has 1000 posts
quittingsmoking has 1000 posts
QuitVaping has 1000 posts
QuittingJUUL has 1000 posts


In [27]:
output_dir = './data/reddit/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
with open(output_dir+'reddit_data.json', 'w') as output:
    json.dump(all_data, output)
df = pd.DataFrame.from_dict(all_data)
df.to_csv(output_dir+'reddit_data.csv', index=False)